# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy 
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/scottbarnes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/scottbarnes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
! ls Disaster

DisasterResponse.db            ML Pipeline Preparation.ipynb
ETL Pipeline Preparation.ipynb


In [2]:
def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql_table('Messages',con=engine)
    X = df['message']
    y = df[df.columns[4:]]
    return X, y

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def model_pipeline():    
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
         ('clf',MultiOutputClassifier(RandomForestClassifier()))
    ])
    parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        #'clf__estimator__n_estimators': [50, 100, 200],
        #'clf__estimator__min_samples_split': [2, 3, 4],
        
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    return cv

In [5]:
model = model_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    model = model_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.score(X_test,y_test))
main()

ValueError: Table Messages not found

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [125]:
y_pred_df = pd.DataFrame(y_pred,columns=y_test.columns)
for col in y_test:
    print(col)
    print(classification_report(y_test[col],y_pred_df[col]))

related
             precision    recall  f1-score   support

          0       0.62      0.37      0.46      1497
          1       0.82      0.93      0.88      5004
          2       0.50      0.09      0.16        53

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5381
          1       0.82      0.37      0.51      1173

avg / total       0.87      0.87      0.85      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.71      0.86      0.78      3802
          1       0.73      0.50      0.59      2752

avg / total       0.72      0.71      0.70      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [165]:
import pickle

In [172]:
pickle.dumps??

In [ ]:
aparameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.